# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298520137269                   -0.85    4.0
  2   -8.300218696139       -2.77       -1.25    1.0
  3   -8.300447763563       -3.64       -1.89    3.2
  4   -8.300462304490       -4.84       -2.75    3.1
  5   -8.300464203285       -5.72       -3.10    3.9
  6   -8.300464410743       -6.68       -3.28    1.1
  7   -8.300464521347       -6.96       -3.41    9.0
  8   -8.300464586224       -7.19       -3.54    1.0
  9   -8.300464628371       -7.38       -3.74    1.0
 10   -8.300464640907       -7.90       -3.96    1.0
 11   -8.300464643686       -8.56       -4.27    1.8


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64164833370                   -0.70    5.6
  2   -16.64595892497       -2.37       -1.14    1.0
  3   -16.67922649016       -1.48       -1.86    3.5
  4   -16.67928120189       -4.26       -2.68    3.4
  5   -16.67928614008       -5.31       -3.00    5.2
  6   -16.67928621301       -7.14       -3.31    1.6
  7   -16.67928622198       -8.05       -3.77    1.0
  8   -16.67928622406       -8.68       -4.49    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32502661236                   -0.56    6.5
  2   -33.33269879507       -2.12       -1.00    1.0
  3   -33.33579066178       -2.51       -1.68    2.8
  4   -33.33687721612       -2.96       -2.39    4.0
  5   -33.33693323320       -4.25       -2.87    9.2
  6   -33.33694384981       -4.97       -3.76    3.4
  7   -33.33694391622       -7.18       -4.19    9.6


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298601723464                   -0.85    4.0
  2   -8.300281261146       -2.77       -1.59    1.0
  3   -8.300449579441       -3.77       -2.55    7.0
  4   -8.300377141258   +   -4.14       -2.29   12.4
  5   -8.300464124741       -4.06       -3.45    3.6
  6   -8.300464594980       -6.33       -3.94    3.2
  7   -8.300464643502       -7.31       -4.89    3.0


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32588965722                   -0.56    6.1
  2   -33.33265896806       -2.17       -1.28    1.0
  3   -33.05514539382   +   -0.56       -1.32    8.0
  4   -30.44051515220   +    0.42       -0.84    5.2
  5   -33.22972226251        0.45       -1.30    5.6
  6   -33.33569998618       -0.97       -2.23    3.6
  7   -33.32326947384   +   -1.91       -1.97    4.1
  8   -33.26552592539   +   -1.24       -1.63    4.9
  9   -33.33665321386       -1.15       -2.65    4.0
 10   -33.33662020035   +   -4.48       -2.50    2.8
 11   -33.33690524955       -3.55       -3.06    2.4
 12   -33.33694012812       -4.46       -3.53    2.4
 13   -33.33694354065       -5.47       -3.96    3.4
 14   -33.33694384252       -6.52       -4.26    4.1


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.